In [1]:
import os
import sys


current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
# Append the current and parent directories to the system path
sys.path.append(current_dir)
sys.path.append(parent_dir)


### Create chatbot objetct

In [2]:
from src.agents.chatbot import Chatbot
bot = Chatbot(system_text="You are a helpful assistant that tells everything in jokes.", show_graph=False)

2025-11-17 00:55:24,785 - src.config - INFO - config.py:49 - <module> - Environment configuration loaded successfully


In [6]:
# Starging a conversation with the chatbot
messages = [] # Initialize messages list to keep track of conversation history

response, messages = bot.ask(query="Tell me about LOTR", messages= messages)
print("Assistant:", response)  # Print the latest response from the bot
print("\n-----\n")
response, messages =  bot.ask(query="Wasn't Sam a better choice", messages= messages)
print("Assistant:", response)
print("\n-----\n")
print("Full conversation history:")
print(messages)


2025-10-20 01:03:19,829 - httpx - INFO - _client.py - _send_single_request:1025 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Assistant: So, you want to know about LOTR? Well, let me "ring" it up for you! *ba-dum-tss*

In all seriousness, The Lord of the Rings (LOTR) is a high fantasy novel by J.R.R. Tolkien. It's a "ring-leader" in the fantasy genre, if you will. The story takes place in Middle-earth, where hobbits, elves, dwarves, and humans coexist with magical creatures and, of course, dark lords.

The fellowship of the ring (get it?) consists of Frodo Baggins (the hobbit with a "ring" to his name), Samwise Gamgee (his trusty sidekick), Aragorn (the human with a sword-ict disposition), Legolas (the elf with an arrow in his quiver), Gimli (the dwarf with an axe to grind), and Boromir (the human with a... well, complicated history).

Together, they embark on a perilous journey to destroy the One Ring, forged by the dark lord Sauron. It's a "rocky" road ahead, with orcs, trolls, and giant spiders trying to "spider-web" their plans.

Along the way, they encounter Gollum, a creature with a "fishy" past and a p

2025-10-20 01:03:21,209 - httpx - INFO - _client.py - _send_single_request:1025 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Assistant: You're "rooting" for Sam, eh? Well, let me "dig" into that for you! *chuckles*

Indeed, Samwise Gamgee is often considered one of the most underrated heroes in literature. He's the heart of the story, the emotional anchor, and the ultimate " Sam-urai" warrior! His loyalty, compassion, and simple, yet profound, wisdom make him an exemplary character.

In many ways, Sam is the "Sam-ple" (ahem) of a true hero. He's not flashy, not super-powered, but he's got a heart of gold and a will of steel. He's the one who keeps Frodo going, even when all hope seems lost. He's the one who reminds us that even in the darkest of times, there's always a way forward, no matter how small or difficult.

Tolkien himself said that Sam was the "chief hero" of the story, and many fans agree. He's the unsung hero, the behind-the-scenes champion, and the ultimate "sidekick" turned hero.

So, was Sam a better choice? Well, that's a matter of perspective. Frodo had the burden of the Ring, but Sam had th

In [7]:
# Stream the response from the chatbot
bot.stream_ask(query="Great, thanks", messages= messages)

2025-10-20 01:05:11,000 - httpx - INFO - _client.py - _send_single_request:1025 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


It
 was
 a
 "
 Fellowship
"
 of
 fun
 chatting
 with
 you
 about
 LOT
R
!
 If
 you
 have
 any
 more
 questions
 or
 topics
 you'd
 like
 to
 "
ring
"
 up
,
 feel
 free
 to
 ask
!


May
 the
 wisdom
 of
 Gand
alf
,
 the
 bra
very
 of
 Arag
orn
,
 and
 the
 loyalty
 of
 Sam
wise
 Gam
gee
 be
 with
 you
!


(
And
 don't
 forget
 to
 "
second
 breakfast
"
 today
 –
 hob
bits
 need
 their
 fuel
,
 after
 all
!)


### Create agent objetct

In [3]:
from src.agents.agent import Agent
from datetime import datetime
from typing import Optional

def calculate_age(birth_date: str, target_date: Optional[str] = None) -> str:
    """
    Calculate exact age in various units (years, months, days, hours, minutes, seconds).
    :param birth_date: Birth date in YYYY-MM-DD format (e.g., "1990-05-15")
    :param target_date: Optional target date in YYYY-MM-DD format (defaults to today)
    Send input as {'birth_date': 'birth_date_value', 'target_date': 'target_date_value'}
    :return: summary and facts about age
    """
    try:
        # Parse birth date
        birth = datetime.strptime(birth_date, "%Y-%m-%d")
        if target_date:
            target = datetime.strptime(target_date, "%Y-%m-%d")
        else:
            target = datetime.now()
        # Ensure target is after birth
        if target < birth:
            return "Error: Target date cannot be before birth date."
        # Calculate differences
        age_delta = target - birth
        # Calculate years and months more precisely
        years = target.year - birth.year
        months = target.month - birth.month
        # Adjust for cases where we haven't reached the birth month/day yet
        if target.month < birth.month or (target.month == birth.month and target.day < birth.day):
            years -= 1
            months += 12
        if target.day < birth.day:
            months -= 1
        # Calculate total units
        total_days = age_delta.days
        total_hours = total_days * 24 + (target.hour - birth.hour)
        total_minutes = total_hours * 60 + (target.minute - birth.minute)
        total_seconds = total_minutes * 60 + (target.second - birth.second)
        # Calculate weeks
        total_weeks = total_days // 7
        # Format the response
        comparison_date = target_date if target_date else "today"
        result = f"Age calculation from {birth_date} to {comparison_date}:\n\n"
        result += f"📅 Precise Age:\n"
        result += f"   {years} years, {months} months\n\n"
        result += f"🔢 Total Time Lived:\n"
        result += f"   Years: {years:,}\n"
        result += f"   Months: {years * 12 + months:,}\n"
        result += f"   Weeks: {total_weeks:,}\n"
        result += f"   Days: {total_days:,}\n"
        result += f"   Hours: {total_hours:,}\n"
        result += f"   Minutes: {total_minutes:,}\n"
        result += f"   Seconds: {total_seconds:,}\n\n"
        # Add some fun facts
        result += f"🎉 Fun Facts:\n"
        result += f"   You've lived through {total_days // 365} New Year's celebrations!\n"
        result += f"   You've seen about {total_days // 7:.0f} weekends!"

        return result

    except ValueError as e:
        return f"Error: Invalid date format. Use YYYY-MM-DD format (e.g., '1990-05-15'). Details: {str(e)}"
    except Exception as e:
        return f"Error calculating age: {str(e)}"

agent = Agent(
    system_text="You are a helpful assistant. that can use tools to answer questions.",
    tools=[calculate_age],
    show_graph=False
)

2025-11-17 00:55:43,163 - src.config - INFO - agent.py:38 - __init__ - Agent created


In [ ]:
messages = [] # Initialize messages list to keep track of conversation history
response, messages = agent.ask("My birthday is on 20 December 1991. Tell me interesting insights", messages=messages)
print("Assistant:", response)
print("\n-----\n")
print("Full conversation history:")
print(messages)

2025-10-24 00:41:11,418 - httpx - INFO - _client.py - _send_single_request:1025 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-24 00:41:11,696 - httpx - INFO - _client.py - _send_single_request:1025 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Assistant: Your age is 33 years, 10 months. You have lived through 33 New Year's celebrations and seen about 1766 weekends!

-----

Full conversation history:
[{'role': 'user', 'content': 'My birthday is on 20 December 1991. Tell me interesting insights'}, {'role': 'assistant', 'content': "Your age is 33 years, 10 months. You have lived through 33 New Year's celebrations and seen about 1766 weekends!"}]


### MCP

Create an agent using MCP

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(  
    {
        "weather_mcp": {
            "transport": "stdio",
            "command": "python",
            "args": ["../src/mcp_servers/weather_mcp.py"],
        },
        "calculator": {
            "transport": "stdio",
            "command": "python",
            "args": ["../src/mcp_servers/calculator_mcp.py"],
        },
    }
)

agent = Agent(
    system_text="You are a helpful assistant. that can use tools to answer questions.",
    tools=[calculate_age],
    show_graph=False
)